In [1]:
!pip install -q transformers datasets evaluate accelerate pillow torchvision scikit-learn huggingface_hub wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 66.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.0/289.0 kB 32.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.6 MB/s eta 0:00:00


In [2]:
import wandb
wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [3]:
from huggingface_hub import notebook_login
notebook_login()

In [33]:
from datasets import load_dataset

dataset = load_dataset("garythung/trashnet")

In [34]:
dataset = dataset["train"].train_test_split(test_size=0.3, shuffle=False)

In [35]:
dataset

DatasetDict({
    train: Dataset({
        features: ['image', 'label'],
        num_rows: 3537
    })
    test: Dataset({
        features: ['image', 'label'],
        num_rows: 1517
    })
})

In [36]:
labels = dataset["train"].features["label"].names
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
  label2id[label] = str(i)
  id2label[str(i)] = label

In [37]:
from transformers import AutoImageProcessor

checkpoint = "google/vit-base-patch16-224-in21k"
image_processor = AutoImageProcessor.from_pretrained(checkpoint)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [38]:
from tensorflow import keras
from tensorflow.keras import layers

size = (image_processor.size["height"], image_processor.size["width"])

train_data_augmentation = keras.Sequential(
    [
        layers.RandomCrop(size[0], size[1]),
        layers.Rescaling(scale=1.0 / 127.5, offset=-1),
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(factor=0.02),
        layers.RandomZoom(height_factor=0.2, width_factor=0.2),
    ],
    name="train_data_augmentation",
)

val_data_augmentation = keras.Sequential(
    [
        layers.CenterCrop(size[0], size[1]),
        layers.Rescaling(scale=1.0 / 127.5, offset=-1),
    ],
    name="val_data_augmentation",
)

In [39]:
import numpy as np
import tensorflow as tf
from PIL import Image


def convert_to_tf_tensor(image: Image):
    np_image = np.array(image)
    tf_image = tf.convert_to_tensor(np_image)
    # `expand_dims()` is used to add a batch dimension since
    # the TF augmentation layers operates on batched inputs.
    return tf.expand_dims(tf_image, 0)


def preprocess_train(example_batch):
    """Apply train_transforms across a batch."""
    images = [
        train_data_augmentation(convert_to_tf_tensor(image.convert("RGB").resize((224,224)))) for image in example_batch["image"]
    ]
    example_batch["pixel_values"] = [tf.transpose(tf.squeeze(image)) for image in images]
    return example_batch


def preprocess_val(example_batch):
    """Apply val_transforms across a batch."""
    images = [
        val_data_augmentation(convert_to_tf_tensor(image.convert("RGB").resize((224,224)))) for image in example_batch["image"]
    ]
    example_batch["pixel_values"] = [tf.transpose(tf.squeeze(image)) for image in images]
    return example_batch

In [40]:
dataset["train"].set_transform(preprocess_train)
dataset["test"].set_transform(preprocess_val)

In [41]:
from transformers import DefaultDataCollator

data_collator = DefaultDataCollator(return_tensors="tf")

In [42]:
import evaluate

accuracy = evaluate.load("accuracy")

In [43]:
import numpy as np


def compute_metrics(eval_pred):
  predictions, labels = eval_pred
  predictions = np.argmax(predictions, axis=1)
  return accuracy.compute(predictions=predictions, references=labels)

In [44]:
from transformers import create_optimizer

batch_size = 32
num_epochs = 3
num_train_steps = len(dataset["train"]) * num_epochs
learning_rate = 3e-5
weight_decay_rate = 0.01

optimizer, lr_schedule = create_optimizer(
    init_lr=learning_rate,
    num_train_steps=num_train_steps,
    weight_decay_rate=weight_decay_rate,
    num_warmup_steps=0,
)

In [45]:
from transformers import TFAutoModelForImageClassification

model = TFAutoModelForImageClassification.from_pretrained(
  checkpoint,
  id2label=id2label,
  label2id=label2id,
)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFViTForImageClassification: ['pooler.dense.bias', 'pooler.dense.weight']
- This IS expected if you are initializing TFViTForImageClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFViTForImageClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFViTForImageClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [46]:
# converting our train dataset to tf.data.Dataset
tf_train_dataset = dataset["train"].to_tf_dataset(
    columns="pixel_values", label_cols="label", shuffle=True, batch_size=batch_size, collate_fn=data_collator
)

# converting our test dataset to tf.data.Dataset
tf_eval_dataset = dataset["test"].to_tf_dataset(
    columns="pixel_values", label_cols="label", shuffle=True, batch_size=batch_size, collate_fn=data_collator
)

In [47]:
from tensorflow.keras.losses import SparseCategoricalCrossentropy

loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer=optimizer, loss=loss)

In [48]:
from transformers.keras_callbacks import KerasMetricCallback, PushToHubCallback

metric_callback = KerasMetricCallback(metric_fn=compute_metrics, eval_dataset=tf_eval_dataset)

In [25]:
wandb.init(
        project="vit-trash-classification",
        config={
            "epoch": 3,
        },
)
config = wandb.config

In [49]:
import random
import numpy as np
import tensorflow as tf
from wandb.integration.keras import WandbMetricsLogger, WandbModelCheckpoint
wandb_callbacks = [
        WandbMetricsLogger(),
        WandbModelCheckpoint(filepath="my_model_{epoch:02d}"),
        metric_callback
]

In [50]:
model.fit(tf_train_dataset, validation_data=tf_eval_dataset, epochs=3, callbacks=wandb_callbacks)

Epoch 1/3
111/111 [==============================] - ETA: 0s - loss: 1.1010 - accuracy: 0.7317

wandb: Adding directory to artifact (./my_model_01)... Done. 25.1s


111/111 [==============================] - 1366s 12s/step - loss: 1.1010 - accuracy: 0.7462 - val_loss: 0.8347 - val_accuracy: 0.7462
Epoch 2/3
111/111 [==============================] - ETA: 0s - loss: 0.3930 - accuracy: 0.9381

wandb: Adding directory to artifact (./my_model_02)... Done. 128.0s


111/111 [==============================] - 1552s 14s/step - loss: 0.3930 - accuracy: 0.8840 - val_loss: 0.4903 - val_accuracy: 0.8840
Epoch 3/3
111/111 [==============================] - ETA: 0s - loss: 0.2221 - accuracy: 0.9729

wandb: Adding directory to artifact (./my_model_03)... Done. 143.3s


111/111 [==============================] - 1601s 14s/step - loss: 0.2221 - accuracy: 0.9572 - val_loss: 0.2650 - val_accuracy: 0.9572


In [51]:
model.evaluate(tf_eval_dataset)

48/48 [==============================] - 90s 2s/step - loss: 0.2650 - accuracy: 0.9572


[0.2650126516819, 0.957152247428894]

In [52]:
model.summary()

Model: "tf_vi_t_for_image_classification_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vit (TFViTMainLayer)        multiple                  85798656  
                                                                 
 classifier (Dense)          multiple                  4614      
                                                                 
Total params: 85803270 (327.31 MB)
Trainable params: 85803270 (327.31 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [53]:
model.push_to_hub("vision-trash")

tf_model.h5:   0%|          | 0.00/343M [00:00<?, ?B/s]

In [54]:
wandb.finish()

epoch/accuracy,▁▇█
epoch/epoch,▁▅█
epoch/learning_rate,█▄▁
epoch/loss,█▂▁
epoch/val_accuracy,▁▆█
epoch/val_loss,█▄▁
epoch/accuracy,0.97286
epoch/epoch,2
epoch/learning_rate,3e-05
epoch/loss,0.22211
epoch/val_accuracy,0.95715


# END